In [1]:
import sumo_rl
import ray
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.env import PettingZooEnv
from torchrl.envs.libs.pettingzoo import PettingZooWrapper


/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-07 16:24:29,667	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-07 16:24:33,001	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
ray.shutdown()
ray.init()

2024-03-07 16:24:35,559	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.10.13
Ray version:,2.9.3


Step #5.00 (0ms ?*RT. ?UPS, TraCI: 102184ms, vehicles TOT 1 ACT 1 BUF 0)                  1 ACT 1 BUF 0)                     


In [3]:
env_pz = ParallelPettingZooEnv(sumo_rl.parallel_env(
            net_file='net/grid4x4/grid4x4.net.xml',
            route_file='net/grid4x4/grid4x4_1.rou.xml',
            use_gui=False,
            num_seconds=5000))

def env_creator(config):
    return env_pz

register_env("SumoEnv", lambda config: env_creator(config))

env_pz.close()


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 46ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds


/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/pettingzoo/utils/conversions.py:132: UserWarning: The base environment `sumo_rl_v0` does not have a `render_mode` defined.
  warnings.warn(


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 92ms, vehicles TOT 0 ACT 0 BUF 0)                     


/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/pettingzoo/utils/conversions.py:144: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(
/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/pettingzoo/utils/conversions.py:158: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(


In [4]:
agents = [a for a in env_pz.get_agent_ids()]
print(agents)
print(env_pz.observation_space['A1'])

['C0', 'D1', 'D0', 'A1', 'B1', 'D3', 'B3', 'C3', 'A2', 'A0', 'C2', 'D2', 'A3', 'B0', 'C1', 'B2']
Box(0.0, 1.0, (33,), float32)


In [5]:
def custom_policy_fn(agent_id):
    return (None, env_pz.observation_space[agent_id], env_pz.action_space[agent_id], {})

def policy_mapping(agent_id, episode, worker, **kwards):
    return agent_id
    

In [6]:
config = {
    "env": "SumoEnv",
    "multiagent": {
        "policies": {
            agent_id: custom_policy_fn(agent_id)
            for agent_id in agents
        },
        "policy_mapping_fn": policy_mapping,
    },
    "framework": "torch",
}

In [7]:
tune.run(
    "PPO",
    config=config,
    stop={"timesteps_total": 1600},
    checkpoint_at_end=True,
    local_dir="/home/sandaruvi/Workspace/Playground/sumo_rl_petting_zoo/ray_results",
)

2024-03-07 16:24:38,914	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/sandaruvi/Applications/anaconda3/envs/torch_pz/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  l

(PPO pid=58470)  Retrying in 1 seconds
(PPO pid=58470) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 44ms, vehicles TOT 0 ACT 0 BUF 0)                     
(RolloutWorker pid=58581)  Retrying in 1 seconds
(RolloutWorker pid=58580)  Retrying in 1 seconds
(RolloutWorker pid=58581) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 54ms, vehicles TOT 0 ACT 0 BUF 0)                     
(RolloutWorker pid=58580) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 53ms, vehicles TOT 0 ACT 0 BUF 0)                     


(PPO pid=58470) Install gputil for GPU system monitoring.


Step #5.00 (0ms ?*RT. ?UPS, TraCI: 2219ms, vehicles TOT 1 ACT 1 BUF 0)                    1 ACT 1 BUF 0)                     
Step #1300.00 (1ms ~= 1000.00*RT, ~89000.00UPS, TraCI: 143ms, vehicles TOT 386 ACT 89 BUF 1 ACT 1 BUF 0)                     
(RolloutWorker pid=58580)  Retrying in 1 seconds [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
Step #5.00 (0ms ?*RT. ?UPS, TraCI: 2212ms, vehicles TOT 1 ACT 1 BUF 0)                    1 ACT 1 BUF 0)                     


(RolloutWorker pid=58581) Warning: Vehicle '354' performs emergency braking on lane ':A1_42_0' with decel=9.00, wished=4.50, severity=1.00, time=1380.00.


...

(RolloutWorker pid=58580) 2024-03-07 16:29:52,936	WARNING env_runner_v2.py:155 -- More than 32000 observations in 2000 env steps for episode 556067461535154005 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.


Step #2000.00 (1ms ~= 1000.00*RT, ~224000.00UPS, TraCI: 673ms, vehicles TOT 907 ACT 224 BU1 ACT 1 BUF 0)                     


(PPO pid=58470) 2024-03-07 16:29:54,461	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
(PPO pid=58470) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/sandaruvi/Workspace/Playground/sumo_rl_petting_zoo/ray_results/PPO_2024-03-07_16-24-39/PPO_SumoEnv_183af_00000_0_2024-03-07_16-24-39/checkpoint_000000)


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,timers
PPO_SumoEnv_183af_00000,64000,"{'ObsPreprocessorConnector_ms': 0.008518695831298828, 'StateBufferConnector_ms': 0.00446319580078125, 'ViewRequirementAgentConnector_ms': 0.10933235287666321}","{'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 64000, 'num_agent_steps_trained': 64000}",{},400,{},-21.77,-41.326,-64.33,10,64000,64000,4000,4000,9.96555,4000,4000,9.96555,0,2,0,0,4000,"{'cpu_util_percent': 27.785288966725044, 'ram_util_percent': 43.13555166374781}","{'A0': -0.06999999999999991, 'A1': -0.13, 'A2': -1.22, 'A3': -0.09000000000000088, 'B0': -0.10999999999999999, 'B1': -0.09999999999999953, 'B2': -0.3400000000000001, 'B3': -0.28, 'C0': -0.020000000000000028, 'C1': -0.49, 'C2': -0.23000000000000004, 'C3': -0.020000000000000018, 'D0': -0.29000000000000004, 'D1': -0.12000000000000005, 'D2': -0.8099999999999992, 'D3': -0.26000000000000045}","{'A0': -1.3290000000000002, 'A1': -1.1699999999999997, 'A2': -5.3660000000000005, 'A3': -3.9779999999999993, 'B0': -1.5829999999999997, 'B1': -1.593, 'B2': -1.6350000000000002, 'B3': -4.304, 'C0': -2.4739999999999998, 'C1': -2.311, 'C2': -3.441, 'C3': -1.4289999999999998, 'D0': -3.1100000000000003, 'D1': -2.105, 'D2': -3.0829999999999997, 'D3': -2.415}","{'A0': -3.37, 'A1': -4.54, 'A2': -20.53, 'A3': -7.63, 'B0': -4.82, 'B1': -3.28, 'B2': -4.06, 'B3': -21.61, 'C0': -10.95, 'C1': -5.0, 'C2': -10.14, 'C3': -2.8, 'D0': -9.56, 'D1': -6.04, 'D2': -7.84, 'D3': -4.6}","{'mean_raw_obs_processing_ms': 7.548208894400761, 'mean_inference_ms': 20.786648747445582, 'mean_action_processing_ms': 1.7219177786556856, 'mean_env_wait_ms': 119.43028957351692, 'mean_env_render_ms': 0.0}","{'training_iteration_time_ms': 401382.827, 'sample_time_ms': 300037.576, 'learn_time_ms': 101315.247, 'learn_throughput': 39.481, 'synch_weights_time_ms': 9.571}"


2024-03-07 16:31:36,510	INFO tune.py:1042 -- Total run time: 417.60 seconds (416.88 seconds for the tuning loop).
